In [1]:
data_var = '2024-02-07'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12057,2024-02-07,Brasil Nbb,19:00,União Corinthians,Brasília,1.35,3.02,154.5,1.85,1.85,-7.5,1.95,1.74,hhtQB3qA,0.740741,0.331126,0.540541,0.540541,0.071867,0.6,0.4,NaN,NaN,282.904,230.089144,0.813312,361.404,136.985391,0.379037,203.196,458.398,111.00,511.52,0.0,0.0,0.0,0.0,0.540443,0.000000,0.080484,-3.37,-0.674,-0.519288,0.477560,0.2,-0.277560,0.56,0.112,18.035714,0.191347,0.1,-0.091347
12058,2024-02-07,Brasil Nbb,19:30,Pato,São Paulo,2.02,1.72,155.5,1.86,1.84,1.5,1.87,1.83,vkxMCqU3,0.495050,0.581395,0.537634,0.543478,0.076445,0.8,1.0,NaN,NaN,224.902,182.784626,0.812730,199.922,73.813253,0.369210,212.418,182.282,125.60,109.06,0.0,0.0,0.0,0.0,0.113440,0.007644,0.015289,3.49,0.698,1.461318,0.477690,0.6,0.122310,0.59,0.118,6.101695,0.549446,0.5,-0.049446
12059,2024-02-07,Eua Nba,00:00,Phoenix Suns,Milwaukee Bucks,1.61,2.41,242.5,1.88,2.04,-4.5,2.09,1.85,80ftD3Uj,0.621118,0.414938,0.531915,0.490196,0.036056,0.4,0.2,NaN,NaN,203.584,49.256889,0.241949,188.772,51.205596,0.271256,167.420,188.588,166.75,208.44,0.0,0.0,0.0,0.0,0.281436,0.057723,0.086145,0.70,0.140,4.357143,0.723997,0.7,-0.023997,-2.33,-0.466,-3.025751,0.686353,0.4,-0.286353
12060,2024-02-07,Eua Nba,21:00,Charlotte Hornets,Toronto Raptors,3.12,1.40,223.5,1.89,2.03,5.5,2.09,1.85,ER4z2jZe,0.320513,0.714286,0.529101,0.492611,0.034799,0.6,0.8,NaN,NaN,497.166,167.217236,0.336341,359.576,157.622462,0.438356,414.646,463.804,598.26,502.00,1.0,0.0,0.0,0.0,0.538152,0.050508,0.086145,-5.00,-1.000,-2.120000,0.302664,0.1,-0.202664,-5.00,-1.000,-0.400000,0.363553,0.2,-0.163553
12061,2024-02-07,Eua Nba,21:00,Washington Wizards,Cleveland Cavaliers,5.08,1.19,233.5,1.88,2.05,9.5,2.09,1.85,MuxTz7Mc,0.196850,0.840336,0.531915,0.487805,0.037187,0.2,0.4,NaN,NaN,441.846,207.164688,0.468862,273.522,112.600615,0.411669,540.786,278.978,588.00,143.91,1.0,0.0,0.0,0.0,0.877399,0.061175,0.086145,-5.00,-1.000,-4.080000,0.278283,0.1,-0.178283,3.48,0.696,0.272989,0.421484,0.5,0.078516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12188,2024-02-07,Islândia Premier League Feminina,16:15,Njardvik F,Grindavik F,1.46,2.59,145.5,1.85,1.85,-6.5,2.00,1.72,GrXp4iQI,0.684932,0.386100,0.540541,0.540541,0.071032,0.6,0.8,NaN,NaN,100.384,10.154104,0.101153,134.316,39.427175,0.293540,96.074,141.720,93.22,124.95,0.0,0.0,0.0,0.0,0.394583,0.000000,0.106446,0.56,0.112,4.107143,0.000000,0.0,0.000000,-0.72,-0.144,-11.041667,0.000000,0.0,0.000000
12189,2024-02-07,Mundo Amistosos Internacionais,13:00,Polónia F,Republica Checa F,1.21,4.13,142.5,1.84,1.86,-10.5,1.84,1.86,fs6Lc4LH,0.826446,0.242131,0.543478,0.537634,0.068577,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.773315,0.007644,0.007644,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12190,2024-02-07,Rússia Copa Da Rússia,13:30,MBA Moscow,Parma Perm,1.11,5.69,147.5,1.80,1.94,-12.5,2.05,1.65,b7KFcY1O,0.900901,0.175747,0.555556,0.515464,0.076648,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,1162.50,0.0,1.0,0.0,0.0,0.952514,0.052938,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12191,2024-02-07,Rússia Copa Da Rússia,13:30,Zenit Petersburg,Samara,1.01,21.00,146.5,1.83,1.91,-23.5,2.00,1.70,2uR6afXB,0.990099,0.047619,0.546448,0.523560,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Phoenix Suns,Milwaukee Bucks,242.5,1.88,1.0000,Over
1,21:00,Eua Nba,Charlotte Hornets,Toronto Raptors,223.5,1.89,1.0000,Over
2,21:00,Eua Nba,Washington Wizards,Cleveland Cavaliers,233.5,1.88,1.0000,Over
3,21:30,Eua Nba,Miami Heat,San Antonio Spurs,223.5,1.94,0.9141,Over
4,21:30,Eua Nba,Philadelphia 76ers,Golden State Warriors,236.5,1.83,1.0000,Over
